With LangChain Reterival Question answering and Embedding Model

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from litellm import completion


c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community import document_loaders

In [3]:
db_name = 'revision_duplicate'
import glob
from pathlib import Path

from langchain_core.documents import Document


def fetch_documents():
  folder = Path("./")
  documents = []
  for folder in folder.iterdir():
    for file in glob.glob(f"{folder}/**.md"):
      doc_type = str(folder)
      print(doc_type)
      with open(file,'r',encoding="utf-8") as f:
        documents.append(Document(
                            page_content=f.read(),
                            metadata={
                                "type": folder.name,
                                "source": file
                            }))
  print(len(documents))
  return documents

In [4]:
documents = fetch_documents()

company
company
company
company
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
products
products
products
products
products
products
products
products
76


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

In [34]:
print(len(chunks[0].page_content))
print(len(chunks[1].page_content))


255
296


In [49]:
type(chunks[0])

__main__.Chunk

In [6]:
import tiktoken 
entire_knowledge_base = ""
folder = Path("./")
for folder in folder.iterdir():
    for file in glob.glob(f"{folder}/**.md"):
      doc_type = str(folder)
      print(doc_type)
      with open(file,'r',encoding="utf-8") as f:
        entire_knowledge_base+= f.read()

print(f"Total characters in knowledge base: {len(entire_knowledge_base):,}")

encoding = tiktoken.encoding_for_model("gpt-4.1-nano")
tokens = encoding.encode(entire_knowledge_base)
token_count = len(tokens)
print(f"Total tokens for text-embedding-3-small: {token_count:,}")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

company
company
company
company
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
products
products
products
products
products
products
products
products
Total characters in knowledge base: 304,282
Total tokens for text-embedding-3-small: 63,530


In [37]:
chunks[1]

Document(metadata={'type': 'company', 'source': 'company\\about.md'}, page_content='The company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 200 employees with 12 offices across the')

In [45]:
embeddings= OpenAIEmbeddings(model="text-embedding-3-small")
import os
if os.path.exists(db_name):
  Chroma(persist_directory = db_name,embedding_function=embeddings).delete_collection()
vectorstore = Chroma.from_documents(documents=chunks,embedding = embeddings,persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 1545 documents


In [ ]:


# vectorstore_colelction = Chroma.get_or_create_collection(db_name) #use it when using Chroma native package
#
collection = vectorstore._collection
count = collection.count()
sample_embedding = collection.get(include=["embeddings"])['embeddings'][0]
print(f"Vectorstore created with {vectorstore._collection.count()} and with the dimensions of {len(sample_embedding)}")

Vectorstore created with 1545 dimensions 1536


NOW with out Langchain

In [7]:
from chromadb import PersistentClient
from pathlib import Path
from pydantic import BaseModel, Field
from tqdm import tqdm
from litellm import completion
from openai import OpenAI


In [8]:
class Metadata(BaseModel):
  type:str
  source:str
class Chunk(BaseModel):
  page_content:str
  metadata:Metadata
class Chunks(BaseModel):
  chunks:list[Chunk]

In [9]:
documents = fetch_documents()

company
company
company
company
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
contracts
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
employees
products
products
products
products
products
products
products
products
76


In [10]:
import json

def make_prompt(document):
  how_many = len(document.page_content)//400+1
  return f"""
  You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document.metadata['type']} make this as an metadatas type
source:{document.metadata['source']} and this as metadatas source

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document.page_content}

Respond with the chunks.
  """

In [11]:
def make_messages(document):
  return [
    {'role':'user','content':make_prompt(document)}
  ]

In [12]:
make_messages(documents[0])

[{'role': 'user',
  'content': "\n  You take a document and you split the document into overlapping chunks for a KnowledgeBase.\n\nThe document is from the shared drive of a company called Insurellm.\nThe document is of type: company make this as an metadatas type\nsource:company\\about.md and this as metadatas source\n\nA chatbot will use these chunks to answer questions about the company.\nYou should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.\nThis document should probably be split into 6 chunks, but you can have more or less as appropriate.\nThere should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.\n\nFor each chunk, you should provide a headline, a summary, and the original text of the chunk.\nTogether your chunks should represent the entire document with overlap.\n\nHere is the

In [13]:
def process_document(document):
  messages = make_messages(document)
  response = completion(model="gpt-4.1-nano",messages=messages,response_format=Chunks)
  reply = response.choices[0].message.content
  doc_as_chunks = Chunks.model_validate_json(reply).chunks
  return [c for c in doc_as_chunks]
  

In [14]:
process_document(documents[0])


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



InternalServerError: litellm.InternalServerError: InternalServerError: OpenAIException - Connection error.

In [15]:
def create_chunk(documents):
  chunks = []
  for doc in tqdm(documents):
    chunks.extend(process_document(doc))
  return chunks

In [16]:
chunks = create_chunk(documents)

100%|██████████| 76/76 [12:12<00:00,  9.63s/it]


In [17]:
len(chunks)

284

In [18]:
chunks[0]

Chunk(page_content="**Insurellm Overview: A Disruptive Insurance Tech Startup**\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.\n\nThis opening segment introduces Insurellm, emphasizing its founding purpose, founder, and initial product. It sets the context for the company's innovative approach within the insurance industry.", metadata=Metadata(type='company', source='company/about.md'))

In [19]:
from openai import OpenAI
openai = OpenAI()
def create_embeddings(chunks):
  chroma = PersistentClient(path= "./revision_no_langchainDB")
  if "docs" in [c.name for c in chroma.list_collections()]:
    chroma.delete_collection("docs")
  
  texts = [chunk.page_content for chunk in chunks]
  emb = openai.embeddings.create(model="text-embedding-3-small",input = texts).data
  vectors = [e.embedding for e in emb]
  collection = chroma.get_or_create_collection("docs")

  ids = [str(i) for i in range(len(chunks)) ]
  metas = [
    {
        "type": chunk.metadata.type,
        "source": chunk.metadata.source
    }
    for chunk in chunks
]


  collection.add(ids=ids,embeddings=vectors,documents=texts,metadatas=metas)

  print(f"Vectorstore created with {collection.count()} documents")
  



In [20]:
create_embeddings(chunks)

Vectorstore created with 284 documents


In [21]:
doc_type = [doc.metadata['type'] for doc in documents if doc.metadata['type'] ]
doc_type=set(doc_type)
doc_type

{'company', 'contracts', 'employees', 'products'}

In [22]:
import plotly.express as ex
from sklearn.manifold import TSNE
import numpy as np

chroma = PersistentClient(path="revision_no_langchainDB")
collection = chroma.get_or_create_collection("docs")
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result["documents"]
metadatas = result['metadatas']


tsne = TSNE(n_components=2,random_state=42)
reduced_vectors = tsne.fit_transform(vectors)
doc_type = [m["type"] for m in metadatas]

hover_text = [f"Text:{t[:100]}" for t in documents]
er = ex.scatter(
  x=reduced_vectors[:,0],
  y=reduced_vectors[:,1],
  color=doc_type,
  hover_data = {"text":hover_text}
)

er.show()

In [23]:
doc_type = [m["type"] for m in metadatas]
chunk_sec = [chunk.metadata.type for chunk in chunks]
set(chunk_sec)

{'company',
 'contract',
 'contract renewal',
 'contract terms',
 'contract_renewal',
 'contract_terms',
 'contracts',
 'employees',
 'features',
 'products',
 'signatures',
 'support'}

In [24]:

tsne = TSNE(n_components=3,random_state=42)
reduced_vectors = tsne.fit_transform(vectors)
doc_type = [m["type"] for m in metadatas]

hover_text = [f"Text:{t[:100]}" for t in documents]
er = ex.scatter_3d(
  x=reduced_vectors[:,0],
  y=reduced_vectors[:,1],
  z=reduced_vectors[:,2],
  color=doc_type,
  hover_data = {"text":hover_text}
)

er.show()

In [25]:
len(chunks)

284

## Rank Order

In [26]:
class RankOrder(BaseModel):
  order:list[int]

In [27]:
def rerank(question,chunks):
  system_prompt ="""
  You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
  """

  user_prompt = f"The user has asked the following question:\n\n{question}\n\nOrder all the chunks of text by relevance to the question, from most relevant to least relevant. Include all the chunk ids you are provided with, reranked.\n\n"
  user_prompt += "Here are the chunks:\n\n"

  for index,chunk in enumerate(chunks):
    user_prompt += f'# Chunk ID: {index}: \n {chunk.page_content} \n'
  user_prompt += "Reply only with the list of ranked chunk ids, nothing else."

  messages = [
    {'role':'system','content':system_prompt},{'role':'user','content':user_prompt}
  ]

  response = completion(model="gpt-4.1-nano",messages=messages,response_format=RankOrder)
  reply = response.choices[0].message.content
  order = RankOrder.model_validate_json(reply).order
  
  return [chunks[i] for i in order]

  
  

In [28]:
RETRIEVAL_K =10
def fetch_context_unranked(question):
  query = openai.embeddings.create(model="text-embedding-3-small",input=[question]).data[0].embedding
  results = collection.query(query_embeddings=[query],n_results = RETRIEVAL_K)
  chunks=[]
  for result in zip(results['documents'][0],results['metadatas'][0]):
    chunks.append(Chunk(page_content=result[0],metadata=result[1]))
  return chunks


In [29]:
question = "Who won the IIOTY award?"
chunks = fetch_context_unranked(question)
for chunk in chunks:
    print(chunk.page_content[:115]+"...")

**Headline:** Priya Sharma's Performance Awards and Ratings

**Summary:** Over the years, Priya has received high-p...
**Headline:** Priya Sharma's Other Achievements, Skills, and Recognition

**Summary:** Priya is a recognized expert...
## Annual Performance History
- **2023:** Rating: 4.9/5
  *Outstanding performance. Successfully led AI transformat...
# HR Record

# Oliver Spencer

## Summary
- **Date of Birth**: May 14, 1990
- **Job Title**: Backend Software Engin...
# HR Record

# Maxine Thompson

## Summary
- **Date of Birth:** January 15, 1991
- **Job Title:** Data Engineer
- *...
**Header: Performance and Compensation Highlights**

Annual Performance Ratings:
- 2023: 4.6/5 - Excellent performa...
**Header: Skills, Recognition, and Other HR Notes**

Skills:
- Expert in Swift, Kotlin, React Native, UI/UX pattern...
# HR Record

# Carlos Rodriguez

## Summary
- **Date of Birth:** October 12, 1989
- **Job Title:** Solutions Engine...
## Insurellm Career Progression
- **April 2020:*

In [ ]:
len(chunks)

10

In [ ]:
for index,chunk in enumerate(chunks):

  if "maxine" in chunk.page_content.lower():
    print(index)
    break

3


In [30]:
def fetch_context(question):
  chunks = fetch_context_unranked(question)
  return rerank(question,chunks)


In [31]:
indexes = fetch_context("Who went to Manchester University?")
indexes


[Chunk(page_content='## Other HR Notes\n- **Education:** BS in Computer Science from University of Manchester\n- **Skills:** Proficient in React, TypeScript, HTML/CSS, Jest for testing. Learning Next.js and GraphQL.\n- **Professional Development:** Completed Advanced React Patterns course (2023). Actively contributes to open-source projects.\n- **Work Style:** Prefers remote work. Strong written communicator. Participates actively in team standups and planning sessions.\n- **Feedback:** Reliable developer with good attention to UI details. Improving technical decision-making skills. Would benefit from more ownership of larger features.', metadata=Metadata(type='employees', source="employees\n't Jessica Liu.md")),
 Chunk(page_content='# HR Record\n\n# Alex Chen\n\n## Summary\n- **Date of Birth:** March 15, 1990\n- **Job Title:** Backend Software Engineer\n- **Location:** San Francisco, California\n- **Current Salary:** $115,000', metadata=Metadata(type='employees', source='employees\\Al

In [32]:
SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""

In [46]:
def make_rag_messages(question,history,chunks):
  context = "\n\n".join(f"Extract from {chunk.metadata.source}:\n{chunk.page_content}" for chunk in chunks)
  system_prompt = SYSTEM_PROMPT.replace("{context}", context)
  return [{'role':'system','content':system_prompt}]+history+[{'role':'user','content':question}]

In [43]:
for index, c in enumerate(indexes):
    if "manchester" in c.page_content.lower():
        print(index)
        

0


In [44]:
print("d")

d


In [36]:
def rewrite_query(question,history=[]):
  message=f"""
  You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
  """
  response = completion(model="gpt-4.1-nano",messages=[{"role":'system','content':message}])
  return response.choices[0].message.content

In [37]:
rewrite_query("Who won the IIOTY award?", [])

'Who was the recipient of the IIOTY award?'

In [47]:
def answer_quetion(question,history):
  query = rewrite_query(question,history)
  print(query)
  chunks = fetch_context(query)
  messages = make_rag_messages(question,history,chunks)
  response = completion(model="gpt-4.1-nano",messages=messages)
  return response.choices[0].message.content,chunks
  

In [48]:
answer_quetion("Who won the IIOTY award?", [])

Who received the IIOTY award?


('Maxine Thompson received the Insurellm IIOTY (Insurellm Innovator of the Year) award in 2023.',
 [Chunk(page_content='## Annual Performance History\n- **2023:** Rating: 4.9/5\n  *Outstanding performance. Successfully led AI transformation initiative. Exceptional strategic leadership and team building. Key driver of company growth.*\n\n- **2022:** Rating: 4.7/5\n  *Exceeded expectations. Strong execution on platform modernization. Improved engineering productivity and velocity significantly.*\n\n- **2021:** Rating: 4.6/5\n  *Excellent performance navigating remote transition. Built strong engineering culture and maintained high delivery standards.*\n\n- **2020:** Rating: 4.4/5\n  *Strong performance during challenging year. Successfully pivoted technology strategy for remote-first world.*\n\n- **2019:** Rating: 4.8/5\n  *Exceptional leadership. Scaled engineering organization effectively while maintaining quality and culture.*\n\n- **2018:** Rating: 4.5/5\n  *Excellent year with succe